In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from generate_attacked import load_model_and_tokenizer


c:\Users\18147\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    try:
        df["logit_diffs"] = [np.array(eval(','.join(row["logit_diffs"].split()))) for _, row in df.iterrows()]
    except:
        df["logit_diffs"] = [np.array(eval(row["logit_diffs"].replace(";", ","))) for _, row in df.iterrows()]
    
    return df

base = load_csv("base_logit_diffs.csv")
adversarial = load_csv("adversarial_logit_diffs.csv")
far_boundary = load_csv("far_boundary_logit_diffs.csv")
very_far_boundary = load_csv("very_far_boundary_logit_diffs.csv")

In [8]:
model, tokenizer = load_model_and_tokenizer()

In [9]:
def print_text(text):
    # Print 100 characters at a time
    for i in range(0, len(text), 100):
        print(text[i:i + 100])

In [10]:
base_text = base["text"][0]
print_text(base_text)

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-a
ppreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Bab
ylon 5 is to Star [[Trek]] (the original). Silly prosthetics, cheap cardboard sets, stilted dialogue
s, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome
 with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci
-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character d
evelopment, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat impo
rtant issues, yet not as a serious philosophy. It's really difficult to care about the characters he
re as they are not simply foolish, just [[missing]] a spark of life. Their actions and reactions are
 [[wooden]] and [[predictable]], often [[painful]] to watch. The makers of Earth KNOW it's 

In [11]:
very_far_boundary_text = very_far_boundary["text"][0]
print_text(very_far_boundary_text)

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-a
ppreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Bab
ylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, C
G that doesn't match the background, and painfully one-dimensional characters cannot be overcome wit
h a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi 
TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character devel
opment, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat importan
t issues, yet not as a serious philosophy. It's really difficult to care about the characters here a
s they are not simply foolish, just missing a spark of life. Their actions and reactions are [[bois]
] and [[prediction]], often painful to watch. The makers of Earth KNOW it's rubbish as they

In [7]:
base_text_words = base_text.split()
very_far_boundary_text_words = very_far_boundary_text.split()

for i in range(len(base_text_words)):
    if base_text_words[i] != very_far_boundary_text_words[i]:
        print(i, base_text_words[i], very_far_boundary_text_words[i])

41 [[Trek]] Trek
140 [[missing]] missing
150 [[wooden]] [[bois]]
152 [[predictable]], [[prediction]],
154 [[painful]] painful
187 as [[como]]
189 [[dull]], [[blunt]],
190 [[cheap]], [[affordable]],
191 poorly [[thinly]]
226 Dallas [[Orlando]]
227 all [[each]]
228 over [[complete]]
229 again. [[anew]].


In [30]:
def compute_logits(sentence, tokenizer, model):
    # Tokenize the sentence
    sentence = sentence.replace("[[", "")
    sentence = sentence.replace("]]", "")
    # print_text(sentence)
    inputs = tokenizer(sentence, return_tensors="pt")
    # Pass the input through the model and return the logits
    outputs = model(**inputs)
    logits = outputs.logits
    return logits.detach().numpy()

base_logits = compute_logits(base_text, tokenizer, model)[0]
adversarial_logits = compute_logits(very_far_boundary_text, tokenizer, model)[0]

print(base_logits)
print(adversarial_logits)

# Calculate confidence
def calculate_confidence(logits):
    e = math.e
    val = e**logits[0] / (e**logits[0] + e**logits[1])
    return max(val, 1 - val)

print(calculate_confidence(base_logits))
print(calculate_confidence(adversarial_logits))

[ 3.3822885 -3.37056  ]
[-2.3013964  2.7441373]
0.9988338123334104
0.9936031592625638


In [31]:
print(base["logit_diffs"][0])

[5.77769756 6.04660416 6.58319664 6.64710903 6.65032148 6.66306496
 6.6677084  6.67013502 6.67248058 6.67253637 6.67423439 6.68019199
 6.68064737 6.68098164 6.68207264 6.68301964 6.68313694 6.68403959
 6.68521023 6.68664551 6.68986034 6.69222403 6.69705963 6.69757891
 6.70051241 6.70139408 6.70419455 6.70605993 6.70838642 6.71185684
 6.71212292 6.71241665 6.71248245 6.71426392 6.71656847 6.71948051
 6.72581291 6.72650385 6.72849655 6.72858906 6.72992897 6.73003244
 6.73034573 6.73211384 6.73234558 6.73271084 6.73281956 6.73404121
 6.73419762 6.73426151 6.73431873 6.73484612 6.73542595 6.73547935
 6.73610735 6.7365818  6.73714828 6.73751068 6.73756361 6.73799324
 6.73933697 6.7394166  6.73977947 6.74065781 6.74093342 6.74157238
 6.74167061 6.7423768  6.74346733 6.74392033 6.74414682 6.74416637
 6.7442627  6.74427795 6.7446537  6.74522161 6.74689579 6.74710178
 6.74739122 6.74757576 6.74803829 6.74851608 6.74885464 6.74925518
 6.74933481 6.74999332 6.75035858 6.75039005 6.75042057 6.7510

In [32]:
print(very_far_boundary["logit_diffs"][0])

[0.92202234 2.4665966  3.01130247 3.0762639  3.46616697 3.69428945
 3.85763788 4.18602228 4.29927015 4.39981222 4.49113321 4.49223232
 4.50160885 4.50984859 4.5418787  4.54559231 4.55217505 4.56865597
 4.63070583 4.63185167 4.63747692 4.64262581 4.64486885 4.64679146
 4.65302801 4.66490841 4.68303251 4.68566084 4.69317722 4.70160961
 4.70260239 4.70447731 4.70591784 4.7105732  4.71914768 4.72178936
 4.72190952 4.74073505 4.74175596 4.75659943 4.77024841 4.77249622
 4.77610111 4.77931213 4.78712368 4.7917347  4.79929304 4.80006552
 4.80870914 4.81116343 4.82390833 4.82551289 4.82672024 4.83745241
 4.83849096 4.83884621 4.84388924 4.84900379 4.85170746 4.85521078
 4.85737324 4.85900402 4.86371613 4.86385632 4.86569881 4.86626339
 4.86891747 4.87744617 4.87863445 4.88466644 4.88695669 4.88777781
 4.89117765 4.89276505 4.89486456 4.89544582 4.89758348 4.90086508
 4.90180779 4.90519762 4.90693188 4.90736055 4.91210651 4.9131937
 4.91356659 4.91534424 4.917768   4.92095375 4.92282867 4.92330

In [33]:
import textattack

In [63]:
dataset = textattack.datasets.HuggingFaceDataset("imdb", split="train")


Reusing dataset imdb (C:\Users\18147\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|██████████| 3/3 [00:00<00:00, 176.47it/s]
textattack: Loading datasets dataset imdb, split train.


In [64]:
dataset.input_columns 

('text',)

In [69]:
# Load log.csv
df = pd.read_csv("very_very_far_boundary.csv")
df.head()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,I [[love]] sci-fi and am willing to put up wit...,I [[loves]] sci-fi and am willing to put up wi...,0.001166,0.996952,0,1,0,561,Failed
1,Worth the entertainment [[value]] of a [[renta...,Worth the entertainment [[valuing]] of a [[lea...,0.007236,0.998512,0,1,0,432,Successful
2,its a [[totally]] average [[film]] with a few ...,its a [[abundantly]] average [[cinematic]] wit...,0.006598,0.991876,0,1,0,281,Failed
3,[[STAR]] [[RATING]]: ***** Saturday Night ****...,[[STARS]] [[RATINGS]]: ***** Saturday Night **...,0.001089,0.998504,0,1,0,894,Successful
4,"First off let me say, If you haven't enjoyed a...","First off let me say, If you haven't enjoyed a...",0.996991,0.998509,1,1,0,15,Successful


In [65]:
# Show first 10 examples
for i in range(10):
    if dataset[i][1] == 1:
        # print(i)
        pass
    print((dataset[i][1]))
    print_text(dataset[i][0]["text"])
        

0
I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it w
hen it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ev
er tried to enter this country, therefore being a fan of films considered "controversial" I really h
ad to see this for myself.<br /><br />The plot is centered around a young Swedish drama student name
d Lena who wants to learn everything she can about life. In particular she wants to focus her attent
ions to making some sort of documentary on what the average Swede thought about certain political is
sues such as the Vietnam War and race issues in the United States. In between asking politicians and
 ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher
, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years a
go, this was considered pornographic. Really, the sex and nudity scenes are few and far b